### Pretrain task

**MLM**

#### Read Feedback Prize - Evaluating Student Writing data

#### Log

* 09.01: do not join the text together, use short text to do MLM task, and reduce max_seq from 2048 to 1024, do not use Feedback Prize dataset
* 09.10: do not join the text together, reduce max_len from 1024 to 512

In [1]:
import os
import re
data_files = os.listdir('./input/pretrain/train')
train_text = []
for data_file in data_files:
    with open('./input/pretrain/train/'+ data_file, 'r') as reader:
        # join all text in a file together
        #train_text += [' '.join(re.split(r'\n+', reader.read()))]
        # do not join the split text segments, use shorter texts for pretraining
        train_text +=re.split(r'\n+', reader.read())

In [2]:
len(train_text)

86707

In [3]:
import random
train_text[random.randint(0,len(train_text))]

'Not only do traditional classes keep distractions to a minimum, they also provide multiple ways of learning material. Students are unique in how they learn, some requiring hands-on experiments while others prefer reading and lectures. Online learning oftentimes is unable to accommodate those unique patterns. From personal experience, hands-on learning is often kept to a minimum, and video-conferencing is limited as connections suffer difficulties resulting in poor audio quality. Students are left to fend for themselves in this scenario because their needs cannot be provided for. If students\' needs aren\'t met, the material is wasted on them as it won\'t "click" in their heads. No learning is being done, so students are not benefiting like they would in an environment designed to meet their learning needs. Students who otherwise wouldn\'t be at school due to a preexisting condition could continue to learn and avoid missing "instruction" time, but they aren\'t really learning much eith

In [4]:
len(train_text[random.randint(0,len(train_text))])

10

#### Read Feedback Prize - English Language Learning data

In [5]:
import pandas as pd
pretrain_df = pd.read_csv('./input/pretrain/train.csv')

In [6]:
list(filter(bool, 'Line 1\n\nLine 3\rLine 4\r\n'.splitlines()))

['Line 1', 'Line 3', 'Line 4']

In [7]:
train_lines = []

for text in pretrain_df.full_text:
    ## join all text in a file together
    train_lines += list(filter(bool, text.splitlines()));
    
    ## do not join the split text segments, use shorter texts for pretraining
    #train_lines += [' '.join(re.split(r'\n+', text))]

print(len(train_lines))    
train_lines += train_text
print(len(train_lines))
train_lines = pd.DataFrame(train_lines, columns = ['train_lines'])

train_lines.to_csv('./input/pretrain/train_MLM_ver4.csv', index = False)

21661
108368


In [8]:
len(train_lines.iloc[random.randint(0,len(train_lines))].values[0].split())

5

In [9]:
import torch
import pandas as pd;
import numpy as np;
import os
from torch.utils.data import Dataset, DataLoader
from torch.utils.checkpoint import checkpoint
from transformers import AutoTokenizer, AutoModelWithLMHead
from transformers import AdamW
from tqdm import tqdm
import os

In [10]:
class CFG:
    seed = 42;
    model_name = 'microsoft/deberta-v3-large'
    epochs = 3;
    batch_size = 32;
    lr = 1e-6;
    weight_decay = 1e-6
    max_len = 256 # use max length as 2048 #01.09, use max_length as 1024
    mask_prob = 0.15;
    n_accumulate = 4
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [11]:
import numpy as np;
import os
def set_seed(seed = CFG.seed):
    np.random.seed(seed);
    torch.manual_seed(seed);
    torch.cuda.manual_seed(seed);
    torch.backends.cudnn.deterministic = True;
    torch.backends.cudnn.benchmark = True;
    os.environ['PYTHONHASHSEED'] = str(seed)
set_seed()

In [12]:
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name);
model = AutoModelWithLMHead.from_pretrained(CFG.model_name);

'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /microsoft/deberta-v3-large/resolve/main/tokenizer_config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7fd32cd25430>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: 6673d0ef-0338-49a6-b46d-139e3c31ad1d)')' thrown while requesting HEAD https://huggingface.co/microsoft/deberta-v3-large/resolve/main/tokenizer_config.json
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/root/miniconda3/lib/python3.8/site-packages/transformers/convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have

In [13]:
special_tokens = tokenizer.encode_plus('[CLS] [SEP] [MASK] [PAD]',
                                      add_special_tokens = False,
                                      return_tensors='pt')
special_tokens = torch.flatten(special_tokens['input_ids'])
special_tokens

tensor([     1,      2, 128000,      0])

In [14]:
def getMaskedLabels(input_ids):
    rand = torch.rand(input_ids.shape);
    mask_arr = (rand < CFG.mask_prob);
    
    for special_token in special_tokens:
        token = special_token.item();
        mask_arr *= (input_ids != token);
    selection = torch.flatten(mask_arr[0].nonzero()).tolist()
    input_ids[selection] = 128000
    
    return input_ids

In [15]:
class MLMDataset:
    def __init__(self, data, tokenizer):
        self.data = data;
        self.tokenizer = tokenizer
        
    def __len__(self):
        return len(self.data);
    
    def __getitem__(self, idx):
        text = self.data[idx]
        
        tokenized_data = self.tokenizer.encode_plus(
                            text,
                            max_length = CFG.max_len,
                            truncation = True,
                            padding = 'max_length',
                            add_special_tokens = True,
                            return_tensors = 'pt'
                        )
        input_ids = torch.flatten(tokenized_data.input_ids);
        attention_mask = torch.flatten(tokenized_data.input_ids);
        labels = getMaskedLabels(input_ids)
        
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels
        }

In [16]:
len(train_lines.train_lines.unique())

101089

In [17]:
train_data = MLMDataset(train_lines.train_lines.unique(), tokenizer)
dataloader = DataLoader(train_data, batch_size = CFG.batch_size, shuffle = True)

In [18]:
len(train_data), len(dataloader)

(101089, 3160)

In [19]:
optimizer = AdamW(model.parameters(), lr = CFG.lr, weight_decay = CFG.weight_decay);

/root/miniconda3/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [20]:
def train_loop(model, device):
    model.train()
    batch_losses = []
    loop = tqdm(dataloader, leave=True)
    for batch_num, batch in enumerate(loop):
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

        loss = outputs.loss
        batch_loss = loss / CFG.n_accumulate
        batch_losses.append(batch_loss.item())
    
        loop.set_description(f"Epoch {epoch + 1}")
        loop.set_postfix(loss=batch_loss.item())
        batch_loss.backward()
        
        if batch_num % CFG.n_accumulate == 0 or batch_num == len(dataloader):
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5)
            optimizer.step()
            model.zero_grad()

    return np.mean(batch_losses)

In [21]:
device = CFG.device
model.to(device)
history = []
best_loss = np.inf
prev_loss = np.inf
model.gradient_checkpointing_enable()
print(f"Gradient Checkpointing: {model.is_gradient_checkpointing}")

for epoch in range(CFG.epochs):
    loss = train_loop(model, device)
    history.append(loss)
    print(f"Loss: {loss}")
    if loss < best_loss:
        print("New Best Loss {:.4f} -> {:.4f}, Saving Model".format(prev_loss, loss))
        # torch.save(model.state_dict(), "./deberta_mlm.pt")
        model.save_pretrained('./input/pretrain/pretrained_model_2109/')
        best_loss = loss
    prev_loss = loss

Gradient Checkpointing: True


Epoch 1: 100%|██████████| 3160/3160 [2:41:01<00:00,  3.06s/it, loss=1.18]  


Loss: 1.5799526847804648
New Best Loss inf -> 1.5800, Saving Model


Epoch 2: 100%|██████████| 3160/3160 [2:41:01<00:00,  3.06s/it, loss=0.796]  


Loss: 0.9484926597038402
New Best Loss 1.5800 -> 0.9485, Saving Model


Epoch 3: 100%|██████████| 3160/3160 [2:41:08<00:00,  3.06s/it, loss=1.17]   


Loss: 0.759340821395192
New Best Loss 0.9485 -> 0.7593, Saving Model


In [ ]:
Gradient Checkpointing: True
Epoch 1: 100%|██████████| 4848/4848 [9:03:47<00:00,  6.73s/it, loss=0.753]  
Loss: 1.3033109624615007
New Best Loss inf -> 1.3033, Saving Model
Epoch 2: 100%|██████████| 4848/4848 [9:03:44<00:00,  6.73s/it, loss=0.404]  
Loss: 0.6164927809336299
New Best Loss 1.3033 -> 0.6165, Saving Model
Epoch 3: 100%|██████████| 4848/4848 [9:03:37<00:00,  6.73s/it, loss=0.359]  
Loss: 0.46205416228314833

In [22]:
model

DebertaV2ForMaskedLM(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 1024, padding_idx=0)
      (LayerNorm): LayerNorm((1024,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-23): 24 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (key_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (value_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-07, elementwise_affine=Tr

### Try to load the model

In [20]:
# from transformers import AutoConfig
# config = AutoConfig.from_pretrained('./input/pretrain/pretrained_model/')

In [22]:
# from transformers import AutoModelForSequenceClassification
# model = AutoModelForSequenceClassification.from_pretrained('./input/pretrain/pretrained_model/', config = config)

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at ./input/pretrain/pretrained_model/ and are newly initialized: ['pooler.dense.weight', 'pooler.dense.bias', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
